In [1]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from skimage import exposure

# Base directory where the main folders are located
base_dir = r"C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\Severities"

# Main folders with their subfolders
main_folders = {
    "Cataract": ["Cataract_low", "Cataract_medium", "Cataract_high"],
    "Diabetic Retinopathy": ["Diabetic Retinopathy_low", "Diabetic Retinopathy_medium", "Diabetic Retinopathy_high"],
    "Normal": [],  # No subfolders for Normal
    "Glaucoma": ["Glaucoma_low", "Glaucoma_medium", "Glaucoma_high"],
}

# Dictionary to store features for all subfolders
features_by_folder = {}

# Function to extract HOG features from a grayscale image
def extract_hog_features(image):
    # Extract HOG features and a visualization image
    hog_features, hog_image = hog(image, visualize=True, pixels_per_cell=(16, 16), cells_per_block=(1, 1), feature_vector=True)
    
    # Rescale the HOG image for visualization (optional)
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
    
    return hog_features, hog_image_rescaled

# Function to read images from a folder and extract HOG features
def read_and_extract_features(folder_path):
    # List all image files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Store HOG features from this folder
    hog_features_list = []

    # Loop through each image and extract HOG features
    for image_file in image_files:
        img_path = os.path.join(folder_path, image_file)
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        if image is not None:
            # Extract HOG features
            hog_features, _ = extract_hog_features(image)

            # Append the HOG features to the list
            hog_features_list.append(hog_features)
    
    return hog_features_list

# Loop through each main folder and its subfolders to extract HOG features
for main_folder, subfolder_list in main_folders.items():
    if subfolder_list:
        # Process each subfolder under the main folder
        for subfolder in subfolder_list:
            print(f"Processing subfolder: {main_folder}/{subfolder}")
            folder_path = os.path.join(base_dir, main_folder, subfolder)

            # Extract features from the current subfolder
            features = read_and_extract_features(folder_path)

            # Store the extracted features in the dictionary
            features_by_folder[f"{main_folder}/{subfolder}"] = features

            # Provide feedback on the number of images processed
            print(f"Extracted HOG features from {len(features)} images in '{main_folder}/{subfolder}'")
    else:
        # If no subfolders, process the main folder directly (like Normal)
        print(f"Processing main folder: {main_folder}")
        folder_path = os.path.join(base_dir, main_folder)

        # Extract features from the current main folder
        features = read_and_extract_features(folder_path)

        # Store the extracted features in the dictionary
        features_by_folder[main_folder] = features

        # Provide feedback on the number of images processed
        print(f"Extracted HOG features from {len(features)} images in '{main_folder}'")

# Now 'features_by_folder' contains HOG features from all specified folders and subfolders.


Processing subfolder: Cataract/Cataract_low
Extracted HOG features from 382 images in 'Cataract/Cataract_low'
Processing subfolder: Cataract/Cataract_medium
Extracted HOG features from 350 images in 'Cataract/Cataract_medium'
Processing subfolder: Cataract/Cataract_high
Extracted HOG features from 300 images in 'Cataract/Cataract_high'
Processing subfolder: Diabetic Retinopathy/Diabetic Retinopathy_low
Extracted HOG features from 317 images in 'Diabetic Retinopathy/Diabetic Retinopathy_low'
Processing subfolder: Diabetic Retinopathy/Diabetic Retinopathy_medium
Extracted HOG features from 401 images in 'Diabetic Retinopathy/Diabetic Retinopathy_medium'
Processing subfolder: Diabetic Retinopathy/Diabetic Retinopathy_high
Extracted HOG features from 380 images in 'Diabetic Retinopathy/Diabetic Retinopathy_high'
Processing main folder: Normal
Extracted HOG features from 1074 images in 'Normal'
Processing subfolder: Glaucoma/Glaucoma_low
Extracted HOG features from 354 images in 'Glaucoma/G

In [2]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model

# Base directory for the dataset
base_dir = r"C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\Severities"

# Main folders with their subfolders
main_folders = {
    "Cataract": ["Cataract_low", "Cataract_medium", "Cataract_high"],
    "Diabetic Retinopathy": ["Diabetic Retinopathy_low", "Diabetic Retinopathy_medium", "Diabetic Retinopathy_high"],
    "Normal": [],  # No subfolders for Normal
    "Glaucoma": ["Glaucoma_low", "Glaucoma_medium", "Glaucoma_high"],
}

# Prepare the data generator for data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,  # Example augmentation settings
    horizontal_flip=True  # Example augmentation setting
)

# Training dataset from specified subfolders
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),  # DenseNet input size
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Validation dataset from specified subfolders
val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),  # DenseNet input size
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Load the pre-trained DenseNet121 model without the top layer (for transfer learning)
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom top layers for our specific problem
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)  # Flattening
x = layers.Dense(256, activation='relu')(x)  # Fully connected layer
x = layers.Dropout(0.5)(x)  # Dropout for regularization
output = layers.Dense(train_generator.num_classes, activation='softmax')(x)  # Output layer for categorical classification

# Create the new model
model = Model(inputs=base_model.input, outputs=output)

# Freeze the base DenseNet layers
for layer in base_model.layers:
    layer.trainable = False  # Use pre-trained weights

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Evaluate the model
loss, accuracy = model.evaluate(val_generator)
print(f"Validation accuracy: {accuracy:.2f}")


Found 3371 images belonging to 4 classes.
Found 840 images belonging to 4 classes.
29084464/29084464 [==============================] - 16s 1us/step
Epoch 1/10
106/106 [==============================] - 1896s 18s/step - loss: 0.8669 - accuracy: 0.6446 - val_loss: 0.6183 - val_accuracy: 0.7440
Epoch 2/10
106/106 [==============================] - 2396s 23s/step - loss: 0.6367 - accuracy: 0.7321 - val_loss: 0.4743 - val_accuracy: 0.8274
Epoch 3/10
106/106 [==============================] - 2439s 23s/step - loss: 0.5755 - accuracy: 0.7603 - val_loss: 0.4748 - val_accuracy: 0.8286
Epoch 4/10
106/106 [==============================] - 2165s 20s/step - loss: 0.5563 - accuracy: 0.7787 - val_loss: 0.3927 - val_accuracy: 0.8548
Epoch 5/10
106/106 [==============================] - 2073s 20s/step - loss: 0.5237 - accuracy: 0.7903 - val_loss: 0.4052 - val_accuracy: 0.8607
Epoch 6/10
106/106 [==============================] - 2420s 23s/step - loss: 0.5173 - accuracy: 0.7909 - val_loss: 0.4456 - va

In [3]:
import os
import tensorflow as tf
from tensorflow import keras

# Path to save the model
save_path = r"C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\DenseNet"

# Save the model
model.save(save_path)
print("Model saved to:", save_path)


INFO:tensorflow:Assets written to: C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\DenseNet\assets


INFO:tensorflow:Assets written to: C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\DenseNet\assets


Model saved to: C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\DenseNet


In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the pre-trained model from a saved file
model_path = r"C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\model.h5"
model = keras.models.load_model(model_path)

# Define the class names based on the subfolder names used during training
class_names = [
    "Cataract_low", "Cataract_medium", "Cataract_high",
    "Diabetic Retinopathy_low", "Diabetic Retinopathy_medium", "Diabetic Retinopathy_high",
    "Normal",
    "Glaucoma_low", "Glaucoma_medium", "Glaucoma_high"
]

# Load and preprocess an image for prediction
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))  # Adjust target size to your model's input
    img_array = image.img_to_array(img)  # Convert the image to an array
    img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension
    img_array /= 255.0  # Rescale to the same range used during training
    return img_array

# Function to make a prediction and return the predicted class
def predict_class(img_path):
    img_array = preprocess_image(img_path)
    prediction = model.predict(img_array)  # Get the prediction
    predicted_class_index = np.argmax(prediction)  # Get the index of the highest probability
    predicted_class_name = class_names[predicted_class_index]  # Map index to class name
    return predicted_class_name

# Example usage
image_path = r"C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\test_image.jpg"
predicted_class = predict_class(image_path)

print(f"The predicted class for the given image is: {predicted_class}")


In [10]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the pre-trained model from a saved file
model_path = r"C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\DenseNet"
model = keras.models.load_model(model_path)

# Define the class names based on the subfolder names used during training
class_names = [
    "Cataract_low", "Cataract_medium", "Cataract_high",
    "Diabetic Retinopathy_low", "Diabetic Retinopathy_medium", "Diabetic Retinopathy_high",
    "Normal",
    "Glaucoma_low", "Glaucoma_medium", "Glaucoma_high"
]

# Load and preprocess an image for prediction
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))  # Adjust target size to your model's input
    img_array = image.img_to_array(img)  # Convert the image to an array
    img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension
    img_array /= 255.0  # Rescale to the same range used during training
    return img_array

# Function to make a prediction and return the predicted class
# Function to make a prediction and return the predicted class along with prediction probabilities
def predict_class(img_path):
    img_array = preprocess_image(img_path)
    prediction = model.predict(img_array)  # Get the prediction
    predicted_class_index = np.argmax(prediction)  # Get the index of the highest probability
    predicted_class_name = class_names[predicted_class_index]  # Map index to class name
    return predicted_class_name, prediction

In [47]:
image_path = r"C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\Test Dataset\_19_9976222.jpg"
predicted_class, prediction_probabilities = predict_class(image_path)

print(f"The predicted class for the given image is: {predicted_class}")
print(f"Prediction probabilities: {prediction_probabilities}")

1/1 [==============================] - 0s 302ms/step
The predicted class for the given image is: Cataract_low
Prediction probabilities: [[8.22453260e-01 5.57376305e-04 1.06499255e-01 7.04900548e-02]]


In [43]:
image_path = r"C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\Test Dataset\10266_right.jpeg"
predicted_class, prediction_probabilities = predict_class(image_path)

print(f"The predicted class for the given image is: {predicted_class}")
print(f"Prediction probabilities: {prediction_probabilities}")

1/1 [==============================] - 0s 308ms/step
The predicted class for the given image is: Cataract_medium
Prediction probabilities: [[0.01709453 0.7209907  0.02365575 0.23825894]]


In [56]:
image_path = r"C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\Test Dataset\10149_left.jpeg"
predicted_class, prediction_probabilities = predict_class(image_path)

print(f"The predicted class for the given image is: {predicted_class}")
print(f"Prediction probabilities: {prediction_probabilities}")

1/1 [==============================] - 0s 325ms/step
The predicted class for the given image is: Diabetic Retinopathy_low
Prediction probabilities: [[0.00235401 0.43688455 0.0414187  0.5193428 ]]


In [158]:
image_path = r"C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\Test Dataset\_33_7463078.jpg"
predicted_class, prediction_probabilities = predict_class(image_path)

print(f"The predicted class for the given image is: {predicted_class}")
print(f"Prediction probabilities: {prediction_probabilities}")

1/1 [==============================] - 0s 309ms/step
The predicted class for the given image is: Cataract_high
Prediction probabilities: [[0.14263579 0.0232724  0.55246204 0.28162974]]


In [235]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Base directory for the test dataset
test_base_dir = r"C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\Test Dataset"

# Load the pre-trained model
model_path = r"C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\DenseNet"
model = keras.models.load_model(model_path)

# Prepare the data generator for test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Test dataset from specified subfolders
test_generator = test_datagen.flow_from_directory(
    test_base_dir,
    target_size=(224, 224),  # Ensure it matches the model's input size
    batch_size=32,
    class_mode='categorical'
)

# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_generator)

print(f"Test accuracy: {accuracy:.2f}")


Found 120 images belonging to 4 classes.
4/4 [==============================] - 27s 5s/step - loss: 0.4853 - accuracy: 0.8167
Test accuracy: 0.82


In [3]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.metrics import classification_report  # For precision, recall, F1, and accuracy

# Base directory for the test dataset
test_base_dir = r"C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\Test Dataset"

# Load the pre-trained model
model_path = r"C:\Users\91790\OneDrive\Desktop\BHAVATHARINI\OCULAR DISEASE\DenseNet"
model = keras.models.load_model(model_path)

# Prepare the data generator for test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Test dataset from specified subfolders
test_generator = test_datagen.flow_from_directory(
    test_base_dir,
    target_size=(224, 224),  # Ensure it matches the model's input size
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Do not shuffle to maintain label order
)

# Get the true labels from the test generator
true_labels = test_generator.classes

# Make predictions using the model
predictions = model.predict(test_generator)
predicted_labels = np.argmax(predictions, axis=1)

# Get class names from the generator
class_names = list(test_generator.class_indices.keys())

# Calculate the classification report with precision, recall, F1, and accuracy
report = classification_report(true_labels, predicted_labels, target_names=class_names)

print("Classification Report:")
print(report)


Found 120 images belonging to 4 classes.
4/4 [==============================] - 25s 5s/step
Classification Report:
                      precision    recall  f1-score   support

            Cataract       0.81      0.83      0.82        30
Diabetic Retinopathy       0.80      0.93      0.86        30
            Glaucoma       0.85      0.77      0.81        30
              Normal       0.81      0.73      0.77        30

            accuracy                           0.82       120
           macro avg       0.82      0.82      0.82       120
        weighted avg       0.82      0.82      0.82       120

